In [1]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import tensorflow as tf
import tensorflow.contrib.slim as slim


def getData():
    with open('process/process_file_8.csv') as csv_file:
        reader = csv.reader(csv_file)
        data = list(reader)

    return data


def preprocessData(data):
    one_hot_encoder = preprocessing.OneHotEncoder()

    processed_data = one_hot_encoder.fit_transform(data[:, 0:2]).toarray()
    processed_data = np.append(processed_data, data[:, 2:6], 1)

    processed_data = preprocessing.normalize(processed_data)
    np.random.shuffle(processed_data)

    return processed_data


def learn(data):
    data = preprocessData(data)
    num_params = data.shape[1] - 1

    X = data[:, 0:num_params]
    Y = data[:, num_params].reshape(-1, 1)

    # Split the data into training and testing sets (70/30)
    train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.30)
    train_opening_price = train_X[:, num_params - 1].reshape(-1, 1)
    test_opening_price = test_X[:, num_params - 1].reshape(-1, 1)

    # Get the initial stock prices for computing the relative cost
    stock_data = tf.placeholder(tf.float32, [None, num_params])
    opening_price = tf.placeholder(tf.float32, [None, 1])
    stock_price = tf.placeholder(tf.float32, [None, 1])

    # Number of neurons in the hidden layer
    n_hidden_1 = 3
    n_hidden_2 = 3

    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden_2, 1]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([1]))
    }

    # Implement dropout to reduce overfitting
    keep_prob_input = tf.placeholder(tf.float32)
    keep_prob_hidden = tf.placeholder(tf.float32)

    # Hidden layers
    input_dropout = tf.nn.dropout(stock_data, keep_prob_input)
    layer_1 = slim.fully_connected(input_dropout, n_hidden_1, biases_initializer=None, activation_fn=tf.nn.relu)
    layer_1_dropout = tf.nn.dropout(layer_1, keep_prob_hidden)
    layer_2 = slim.fully_connected(input_dropout, n_hidden_1, biases_initializer=None, activation_fn=tf.nn.relu)
    layer_2_dropout = tf.nn.dropout(layer_2, keep_prob_hidden)
    # regression  layer = (w'x+b)

    output_layer = tf.add(tf.matmul(layer_2_dropout, weights['out']), biases['out'])

    learning_rate = 1e-4
    cost_function = tf.reduce_mean(tf.pow(tf.div(tf.subtract(stock_price, output_layer), opening_price), 2))
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost_function)

    last_cost = 0
    tolerance = 1e-6
    epochs = 1
    max_epochs = 1e6

    config = tf.ConfigProto(log_device_placement = True)
    sess = tf.Session(config=config)
    with sess:
        init = tf.global_variables_initializer()
        sess.run(init)

        history = []
        while True:
            sess.run(optimizer,
                     feed_dict={stock_data: train_X, opening_price: train_opening_price, stock_price: train_Y,
                                keep_prob_input: 0.8, keep_prob_hidden: 0.5})

            if epochs % 100 == 0:
                cost = sess.run(cost_function, feed_dict={stock_data: train_X, opening_price: train_opening_price,
                                                          stock_price: train_Y, keep_prob_input: 0.8,
                                                          keep_prob_hidden: 0.5})
                history.append(cost)
                print("Epoch: %d: Error: %f" % (epochs, cost))

                if abs(cost - last_cost) <= tolerance or epochs > max_epochs:
                    print("Converged.")
                    break
                last_cost = cost

            epochs += 1

        print("Test error: ", sess.run(cost_function, feed_dict={stock_data: test_X, opening_price: test_opening_price,
                                                                 stock_price: test_Y, keep_prob_input: 1.0,
                                                                 keep_prob_hidden: 1.0}))
        test_results = sess.run(output_layer, feed_dict={stock_data: test_X, stock_price: test_Y, keep_prob_input: 1.0,
                                                         keep_prob_hidden: 1.0})

    avg_perc_error = 0
    max_perc_error = 0
    mei = 0
    for i in range(len(test_Y)):
        actual_change = abs(test_Y[i][0] - test_X[i][num_params - 1]) / test_X[i][num_params - 1]
        predicted_change = abs(test_results[i][0] - test_X[i][num_params - 1]) / test_X[i][num_params - 1]
        delta = abs(actual_change - predicted_change)
        avg_perc_error = avg_perc_error + delta
        if delta > max_perc_error:
            max_perc_error = delta
            mei = i

    avg_perc_error = (avg_perc_error * 100) / len(test_Y)
    max_perc_error *= 100
    print("Maximum percentage error: %f\nAverage percentage error: %f\n" % (max_perc_error, avg_perc_error))


def main():
    data = np.array(getData())
    learn(data)


main()


W0306 15:30:17.255444 13692 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0306 15:30:29.122478 13692 deprecation.py:506] From <ipython-input-1-e0db1fbca056>:63: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0306 15:30:29.134446 13692 deprecation.py:323] From c:\ProgramData\Anaconda3\envs\tf_gpu-1.15\lib\site-packages\tensorflow_core\contrib\layers\python\layers\layers.py:1866: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be re

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1

Epoch: 100: Error: 11.644104
Epoch: 200: Error: 11.542737
Epoch: 300: Error: 11.207150
Epoch: 400: Error: 10.923779
Epoch: 500: Error: 10.682984
Epoch: 600: Error: 10.451481
Epoch: 700: Error: 10.207722
Epoch: 800: Error: 9.995956
Epoch: 900: Error: 9.775217
Epoch: 1000: Error: 9.605818
Epoch: 1100: Error: 9.382044
Epoch: 1200: Error: 9.171861
Epoch: 1300: Error: 8.989158
Epoch: 1400: Error: 8.821529
Epoch: 1500: Error: 8.619400
Epoch: 1600: Error: 8.469907
Epoch: 1700: Error: 8.250780
Epoch: 1800: Error: 8.061043
Epoch: 1900: Error: 7.919940
Epoch: 2000: Error: 7.757402
Epoch: 2100: Error: 7.608390
Epoch: 2200: Error: 7.423360
Epoch: 2300: Error: 7.313856
Epoch: 2400: Error: 7.138878
Epoch: 2500: Error: 7.024891
Epoch: 2600: Error: 6.885193
Epoch: 2700: Error: 6.750495
Epoch: 2800: Error: 6.622130
Epoch: 2900: Error: 6.498282
Epoch: 3

Epoch: 27300: Error: 0.007559
Epoch: 27400: Error: 0.006954
Epoch: 27500: Error: 0.006271
Epoch: 27600: Error: 0.005702
Epoch: 27700: Error: 0.005263
Epoch: 27800: Error: 0.004737
Epoch: 27900: Error: 0.004401
Epoch: 28000: Error: 0.004045
Epoch: 28100: Error: 0.003733
Epoch: 28200: Error: 0.003508
Epoch: 28300: Error: 0.003284
Epoch: 28400: Error: 0.003140
Epoch: 28500: Error: 0.002956
Epoch: 28600: Error: 0.002899
Epoch: 28700: Error: 0.002604
Epoch: 28800: Error: 0.002038
Epoch: 28900: Error: 0.002205
Epoch: 29000: Error: 0.002170
Epoch: 29100: Error: 0.001238
Epoch: 29200: Error: 0.001880
Epoch: 29300: Error: 0.001821
Epoch: 29400: Error: 0.001064
Epoch: 29500: Error: 0.001818
Epoch: 29600: Error: 0.001672
Epoch: 29700: Error: 0.001299
Epoch: 29800: Error: 0.001574
Epoch: 29900: Error: 0.001509
Epoch: 30000: Error: 0.001953
Epoch: 30100: Error: 0.001210
Epoch: 30200: Error: 0.001952
Epoch: 30300: Error: 0.001104
Epoch: 30400: Error: 0.000987
Epoch: 30500: Error: 0.000966
Epoch: 306